In [932]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# Import the data set

In [933]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data Cleaning

In [934]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data cleaning
erroneous = lab_data[lab_data['Thorax_length'] == '.']
erroneous

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
253,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,6,3,female,.,1.65,0.573,1.415,1.988,1.986,0.898,1.207,1.003,.


In [935]:
similar_rows = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Gogango_Creek')
    & (lab_data['Sex'] == 'female')
    & (lab_data['Thorax_length'] != ".")
]
similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])
similar_rows.head()

/var/folders/j_/9f1md2vd0_s6kpn12szf1t2c0000gn/T/ipykernel_20440/3776846164.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])


,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
162,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,1,female,1.166,1.910,0.578,1.675,2.254,2.252,0.982,1.381,1.123,1.932
164,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,2,female,1.174,1.829,0.622,1.549,2.170,2.170,0.997,1.360,1.117,1.848
166,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,3,female,1.199,1.864,0.585,1.659,2.244,2.242,1.009,1.376,1.142,1.870
167,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,1,female,1.186,1.872,0.628,1.571,2.199,2.199,1.002,1.356,1.143,1.854
169,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,2,female,1.176,1.872,0.630,1.579,2.207,2.207,0.966,1.345,1.103,1.876


In [936]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

In [937]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

In [938]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [939]:
erroneous_2 = lab_data[(lab_data['l3d'] == 0) & (lab_data['l3p'] != 0)]
erroneous_2

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
61,D._aldrichi,Binjour,-25.52,151.45,1994,1994,25,3,1,female,1.106,0.0,0.6,0.0,0.0,0.0,0.0,1.252,0.0,0.0


In [940]:
similar_rows_2 = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Binjour')
    & (lab_data['Sex'] == 'female')
    & (lab_data['l3d'] != "0")
]

similar_rows_2_l2_median = similar_rows_2['l2'].median()
similar_rows_2_l3d_median = similar_rows_2['l3d'].median()
similar_rows_2_lpd_median = similar_rows_2['lpd'].median()
similar_rows_2_l3_median = similar_rows_2['l3'].median()
similar_rows_2_w1_median = similar_rows_2['w1'].median()
similar_rows_2_w3_median = similar_rows_2['w3'].median()
lab_data.loc[61, 'l2'] = similar_rows_2_l2_median
lab_data.loc[61, 'l3d'] = similar_rows_2_l3d_median
lab_data.loc[61, 'lpd'] = similar_rows_2_lpd_median
lab_data.loc[61, 'l3'] = similar_rows_2_l3_median
lab_data.loc[61, 'w1'] = similar_rows_2_w1_median
lab_data.loc[61, 'w3'] = similar_rows_2_w3_median
lab_data.loc[61, 'wing_loading'] = lab_data.loc[61]['l3'] / lab_data.loc[61]['Thorax_length']
lab_data.loc[[61]]

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
61,D._aldrichi,Binjour,-25.52,151.45,1994,1994,25,3,1,female,1.106,1.799,0.6,1.5235,2.1195,2.1185,0.9235,1.252,1.0495,1.915461


In [941]:
erroneous_3 = lab_data[(lab_data['l3d'] == 0) & (lab_data['l3p'] == 0)]
erroneous_3

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
698,D._aldrichi,Wahruna,-25.2,151.17,1994,1994,20,5,3,female,1.151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [942]:
#TODO: 
similar_rows_3 = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Wahruna')
    & (lab_data['Sex'] == 'female')
    & (lab_data['l3d'] != "0")
]
len(similar_rows_3)

84

In [943]:
similar_rows_3_l2_median = similar_rows_3['l2'].median()
similar_rows_3_l3p_median = similar_rows_3['l3p'].median()
similar_rows_3_l3d_median = similar_rows_3['l3d'].median()
similar_rows_3_lpd_median = similar_rows_3['lpd'].median()
similar_rows_3_l3_median = similar_rows_3['l3'].median()
similar_rows_3_w1_median = similar_rows_3['w1'].median()
similar_rows_3_w2_median = similar_rows_3['w2'].median()
similar_rows_3_w3_median = similar_rows_3['w3'].median()


lab_data.loc[698, 'l2'] = similar_rows_3_l2_median
lab_data.loc[698, 'l3p'] = similar_rows_3_l3p_median
lab_data.loc[698, 'l3d'] = similar_rows_3_l3d_median
lab_data.loc[698, 'lpd'] = similar_rows_3_lpd_median
lab_data.loc[698, 'l3'] = similar_rows_3_l3_median
lab_data.loc[698, 'w1'] = similar_rows_3_w1_median
lab_data.loc[698, 'w2'] = similar_rows_3_w2_median
lab_data.loc[698, 'w3'] = similar_rows_3_w3_median
lab_data.loc[698, 'wing_loading'] = lab_data.loc[698]['l3'] / lab_data.loc[698]['Thorax_length']

lab_data.loc[[698]]

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
698,D._aldrichi,Wahruna,-25.2,151.17,1994,1994,20,5,3,female,1.151,1.7405,0.6,1.4925,2.0855,2.084,0.943,1.2805,1.0645,1.810599


# Model Building

In [944]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [945]:
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
0,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,female,1.238,2.017,0.659,1.711,2.370,2.370,1.032,1.441,1.192,1.914
1,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,male,1.113,1.811,0.609,1.539,2.148,2.146,0.938,1.299,1.066,1.928
2,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,female,1.215,1.985,0.648,1.671,2.319,2.319,0.991,1.396,1.142,1.908
3,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,male,1.123,1.713,0.596,1.495,2.091,2.088,0.958,1.286,1.062,1.860
4,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,2,1,female,1.218,1.938,0.641,1.658,2.298,2.298,1.010,1.418,1.148,1.886


In [946]:
lab_data = shuffle(lab_data)
lab_data['class'] = lab_data.apply(
    lambda x: x['Sex'],
    axis=1
)
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class
361,D._aldrichi,Grandchester,-27.68,152.45,1994,1994,20,5,1,female,...,1.890,0.668,1.578,2.246,2.245,1.018,1.369,1.161,1.849,female
864,D._buzzatii,Binjour,-25.52,151.45,1994,1994,20,5,1,male,...,1.814,0.611,1.492,2.103,2.103,0.925,1.256,1.101,1.866,male
1609,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,20,10,2,female,...,1.967,0.648,1.676,2.325,2.324,1.064,1.462,1.184,1.900,female
1604,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,20,9,2,male,...,1.851,0.632,1.540,2.173,2.172,0.944,1.340,1.055,2.027,male
336,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,30,10,2,male,...,1.466,0.557,1.201,1.758,1.757,0.817,1.069,0.933,1.698,male


In [947]:

train, test = train_test_split(lab_data, test_size=0.2)

In [948]:
number_columns = train.select_dtypes(include=['int64', 'float64']).columns

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(train[number_columns])

train[number_columns] = scaler.transform(train[number_columns])
test[number_columns] = scaler.transform(test[number_columns])

In [949]:
print(number_columns)
train.info()

Index(['Latitude', 'Longitude', 'Year_start', 'Year_end', 'Temperature',
       'Vial', 'Replicate', 'Thorax_length', 'l2', 'l3p', 'l3d', 'lpd', 'l3',
       'w1', 'w2', 'w3', 'wing_loading'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 1384 entries, 1379 to 1596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Species        1384 non-null   object 
 1   Population     1384 non-null   object 
 2   Latitude       1384 non-null   float64
 3   Longitude      1384 non-null   float64
 4   Year_start     1384 non-null   float64
 5   Year_end       1384 non-null   float64
 6   Temperature    1384 non-null   float64
 7   Vial           1384 non-null   float64
 8   Replicate      1384 non-null   float64
 9   Sex            1384 non-null   object 
 10  Thorax_length  1384 non-null   float64
 11  l2             1384 non-null   float64
 12  l3p            1384 non-null   float64
 13  l3d            

In [950]:
classifier = LogisticRegression(penalty='l2', multi_class='multinomial')

classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(multi_class='multinomial')

In [951]:
train['class_predictions'] = classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [952]:
train.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class,class_predictions
1379,D._buzzatii,Oxford_Downs,1.000000,0.000000,0.0,0.0,0.0,0.111111,0.0,female,...,0.706714,0.815287,0.833127,0.833127,0.773529,0.755382,0.728929,0.700210,female,female
1189,D._buzzatii,Gogango_Creek,0.661591,0.366667,0.0,0.0,1.0,0.888889,1.0,male,...,0.250883,0.240446,0.228677,0.229913,0.173529,0.189824,0.175399,0.312369,male,male
1363,D._buzzatii,Grandchester,0.000000,1.000000,0.0,0.0,1.0,0.888889,0.5,female,...,0.367491,0.429936,0.416564,0.416564,0.344118,0.348337,0.302961,0.461216,female,male
118,D._aldrichi,Binjour,0.365482,0.722222,0.0,0.0,1.0,0.222222,0.0,male,...,0.042403,0.388535,0.271941,0.271941,0.176471,0.240705,0.141230,0.540881,male,male
403,D._aldrichi,Grandchester,0.000000,1.000000,0.0,0.0,0.5,0.111111,0.0,male,...,0.395760,0.539809,0.512979,0.511743,0.455882,0.360078,0.480638,0.482180,male,female


In [953]:
#Correct predictions
train.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()

1132

In [954]:
#Incorrect predictions
train.apply(
    lambda x: 1 if x['class_predictions'] != x['class'] else 0,
    axis=1
).sum()

252

In [955]:
# Retry with different parameters and regularization


In [956]:
strong_regularised_classifier = LogisticRegression(C=1e-2, solver='newton-cg', multi_class='multinomial')

strong_regularised_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.01, multi_class='multinomial', solver='newton-cg')

In [957]:
medium_regularised_classifier = LogisticRegression(C=1, solver='newton-cg', multi_class='multinomial')

medium_regularised_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, multi_class='multinomial', solver='newton-cg')

In [958]:
weak_regularised_classifier = LogisticRegression(C=1e2, solver='newton-cg', multi_class='multinomial')

weak_regularised_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100.0, multi_class='multinomial', solver='newton-cg')

In [959]:
l2_regularised_classifier = LogisticRegression(penalty='l2', multi_class='multinomial')

l2_regularised_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(multi_class='multinomial')

In [960]:
non_regularised_classifier = LogisticRegression(penalty=None, multi_class='multinomial', max_iter=20)

non_regularised_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=20, multi_class='multinomial', penalty=None)

In [961]:
test['non_regularised_softmax_predictions'] = non_regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

correct_non_regularised_classification_test = test.apply(
    lambda x: 1 if x['non_regularised_softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()


In [962]:
print(f"accuracy: {correct_non_regularised_classification_test / len(test)}")

accuracy: 0.8155619596541787


In [963]:
test['softmax_predictions'] = classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['strong_regularised_softmax_predictions'] = strong_regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['medium_regularised_softmax_predictions'] = medium_regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['weak_regularised_softmax_predictions'] = weak_regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['non_regularised_softmax_predictions'] = non_regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['logistic_regression_predictions'] = non_regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

correct_softmax_classification_test = test.apply(
    lambda x: 1 if x['softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()

correct_strong_regularised_classification_test = test.apply(
    lambda x: 1 if x['strong_regularised_softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()

correct_medium_regularised_classification_test = test.apply(
    lambda x: 1 if x['medium_regularised_softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()

correct_weak_regularised_classification_test = test.apply(
    lambda x: 1 if x['weak_regularised_softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()

correct_non_regularised_classification_test = test.apply(
    lambda x: 1 if x['non_regularised_softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()


In [964]:
print(f"accuracy: {correct_non_regularised_classification_test / len(test)}")

accuracy: 0.8155619596541787


In [965]:
print(f"accuracy: {correct_softmax_classification_test / len(test)}")

accuracy: 0.8097982708933718


In [966]:
print(f"accuracy: {correct_strong_regularised_classification_test / len(test)}")

accuracy: 0.69164265129683


In [967]:
print(f"accuracy: {correct_medium_regularised_classification_test / len(test)}")

accuracy: 0.8126801152737753


In [968]:
print(f"accuracy: {correct_weak_regularised_classification_test / len(test)}")

accuracy: 0.8213256484149856
